## Zipcode Demographics Generator

By Shirsho Dasgupta (2021)

#### Notes: 

The code combines zipcode-level demographic data (total population, racial and ethnic breakdowns) and economic data (median income, poverty levels) and creates a master spreadsheet for any given state. 

This is done in two steps.

Step 1: Imports the latest zipcode-level data from Census Reporter and creates a spreadsheet.

Step 2: Runs the same spreadsheet through the "uszipcode" pandas library to update any missing figure that the library may have that Census Reporter does not. 

It can then be matched with any data that comes at the zipcode level. 

The raw datasets that are imported to this dataframe are in folder "imports".

A sample spreadsheet generated by this code is attached. See zipcode_finalmaster.csv

The example in this code is with COVID-19 Restaurant Relief Funds for Florida. 

A sample spreadsheet created after the join is attached. See rrf_zipcode_master.csv



##### A few things to remember:


1. This zipcode demographics generator can be made for any state and ready (and up-to-date) code saves a lot of time that would otherwise be sent making countless joins and cleaning up the resulting spreadsheets.


2. The United States Census Bureau does not record data at the zipcode level. Zipcodes are assigned by the United States Postal Service and sometimes change. They may also overlap between counties or even states. The Census Bureau records data using ZCTAs or Zipcode Census Tract Areas that may or may not correspond to zipcodes. To create a generator from scratch i.e., without relying on readymade libraries (with estimations of zipcode-level data) on R/Python or Census Reporter, it is better to consult professional demographers. 


3. The Restaurant Relief Funds spreadsheet imported in the code below (as an example) is a pivot table from the raw data released by the United States Treasury Department's Small Business Administration. 


4. The code used the RRF spreadsheet as a demonstration of how the data can be used. For a true analysis the total amounts of grants received or number of restaurants that got the funds must also be controlled for the number of eateries in that zipcode. One way of doing this is to use the Census Bureau's Zipcode Business Profiles, which is among the few things the bureau records at the zipcode-level. The ZBPs have minimum numbers for employees and establishments in a given zipcode. For eateries, restaurants and bars, the relevant NAICS code is 722---.


5. Hispanic/Latino are ethnicities, not races. 


6. Different people use slightly different Census tables to measure racial breakups. It is better to consult demographers or academics.

#### Sources:

###### Census Reporter
[Total population](https://censusreporter.org/tables/B02001/)

[Black population](https://censusreporter.org/tables/B02009/)

[White population](https://censusreporter.org/tables/B02008/)

[Hispanic/Latino population](https://censusreporter.org/tables/B03003/)
    
[Median family income](https://censusreporter.org/tables/B19113/)
    
[Poverty level](https://censusreporter.org/tables/B17020/)
    
    
###### US Zipcode Pandas Library
Read more [here](https://uszipcode.readthedocs.io/index.html)
    

###### SBA COVID-19 Restaurant Relief Funds
Find the raw data [here](https://data.sba.gov/dataset/rrf-foia/resource/611f26df-0de0-427d-adb1-3bb91c8e8846?inner_span=True)

### Importing libraries

In [1]:
import pandas as pd
import re
import time

### Importing spreadsheets as dataframes

In [2]:
## imports zipcode list for Florida
zip_list = pd.read_csv("fl_zipcodes.csv")

## imports total population for each Florida zipcode
zip_totals = pd.read_csv("totals_B02001.csv")

## imports Black population for each Florida zipcode
zip_black = pd.read_csv("black_B02009.csv")

## imports white population for each Florida zipcode
zip_white = pd.read_csv("white_B02008.csv")

## imports Latino/Hispanic population for each Florida zipcode
zip_latino = pd.read_csv("hispanic_latino_B03003.csv")

## imports median family income for each Florida zipcode
zip_medianIncome = pd.read_csv("medianfamilyincome_B19113.csv")

## imports population below poverty line for each Florida zipcode
zip_poverty = pd.read_csv("povertystatus_B17020.csv")

### Joining dataframes to create master spreadsheet with zipcode details

In [3]:
zipcode_totals = pd.merge(zip_list, zip_totals, on = "Zipcode", how = "left")
zipcode_totals["Total"].fillna("N/A", inplace = True)

In [4]:
zipcode_black = pd.merge(zipcode_totals, zip_black, on = "Zipcode", how = "left")
zipcode_black["Black"].fillna("N/A", inplace = True)

In [5]:
zipcode_white = pd.merge(zipcode_black, zip_white, on = "Zipcode", how = "left")
zipcode_white["White"].fillna("N/A", inplace = True)

In [6]:
zipcode_latino = pd.merge(zipcode_white, zip_latino, on = "Zipcode", how = "left")
zipcode_latino["NotHisp_Latino"].fillna("N/A", inplace = True)
zipcode_latino["Hisp_Latino"].fillna("N/A", inplace = True)

In [7]:
zipcode_medianIncome = pd.merge(zipcode_latino, zip_medianIncome, on = "Zipcode", how = 'left')
zipcode_medianIncome["Median_Income"].fillna("N/A", inplace = True)

In [8]:
zipcode_poverty = pd.merge(zipcode_medianIncome, zip_poverty, on = "Zipcode", how ='left')
zipcode_poverty["Below_Poverty_Percent"].fillna("N/A", inplace = True)
zipcode_poverty["Below_Poverty_Total"].fillna("N/A", inplace = True)

In [9]:
## stores last join as master dataframe
zipcode_master = zipcode_poverty

## displays sample from master dataframe
zipcode_master.head(10)

,Zipcode,Total,Black,White,NotHisp_Latino,Hisp_Latino,geoid,Median_Income,Below_Poverty_Percent,Below_Poverty_Total
0,32003,28676,2248,25726,26761,1915,86000US32003,111651,3.81,1089
1,32004,N/A,N/A,N/A,N/A,N/A,NaN,N/A,N/A,N/A
2,32006,N/A,N/A,N/A,N/A,N/A,NaN,N/A,N/A,N/A
3,32007,N/A,N/A,N/A,N/A,N/A,NaN,N/A,N/A,N/A
4,32008,4425,59,4225,3997,428,86000US32008,42172,12.02,529
5,32009,3661,50,3633,3544,117,86000US32009,90500,4.22,154
6,32011,14509,567,13914,14395,114,86000US32011,71701,9.28,1338
7,32013,N/A,N/A,N/A,N/A,N/A,NaN,N/A,N/A,N/A
8,32024,19452,1083,17863,18063,1389,86000US32024,74137,8.35,1617
9,32025,23416,5751,17053,21525,1891,86000US32025,59197,14.07,2665


In [10]:
## handles null values
zipcode_master["Total"].fillna("N/A", inplace = True)
zipcode_master["Black"].fillna("N/A", inplace = True)
zipcode_master["White"].fillna("N/A", inplace = True)
zipcode_master["NotHisp_Latino"].fillna("N/A", inplace = True)
zipcode_master["Hisp_Latino"].fillna("N/A", inplace = True)
zipcode_master["geoid"].fillna("N/A", inplace = True)
zipcode_master["Median_Income"].fillna("N/A", inplace = True)
zipcode_master["Below_Poverty_Percent"].fillna("N/A", inplace = True)
zipcode_master["Below_Poverty_Total"].fillna("N/A", inplace = True)

## displays first ten rows
zipcode_master.head(10)

,Zipcode,Total,Black,White,NotHisp_Latino,Hisp_Latino,geoid,Median_Income,Below_Poverty_Percent,Below_Poverty_Total
0,32003,28676,2248,25726,26761,1915,86000US32003,111651,3.81,1089
1,32004,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
2,32006,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
3,32007,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
4,32008,4425,59,4225,3997,428,86000US32008,42172,12.02,529
5,32009,3661,50,3633,3544,117,86000US32009,90500,4.22,154
6,32011,14509,567,13914,14395,114,86000US32011,71701,9.28,1338
7,32013,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
8,32024,19452,1083,17863,18063,1389,86000US32024,74137,8.35,1617
9,32025,23416,5751,17053,21525,1891,86000US32025,59197,14.07,2665


### Importing libraries

In [11]:
## imports uszipcode pandas library
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode = False)

In [12]:
## sets search for Florida
res = search.by_state("Florida")
zipcode = res[0]

### Running loop on dataframe to search for data on zipcodes with no values

In [13]:
## displays rows and columns for dataframe
print(zipcode_master.shape)

(1472, 10)


In [14]:
## creates new column to record which row has been updated
zipcode_master["Updated"] = " "

## for loop runs through each row
for row in range(0, 1472):
    
    ## sets zipcode to be searched
    zipcode_search = zipcode_master["Zipcode"][row]
    zipcode = search.by_zipcode(zipcode_search)
    
    ## if the row has null values, the data from the search is stored
    if zipcode_master["Total"][row] == "N/A":
        
        ## try/except block handles exceptions
        try: 
            
            ## stores the data
            pop_total = zipcode.population
            white_total = zipcode.population_by_race[0]['values'][0]['y']
            black_total = zipcode.population_by_race[0]['values'][1]['y']
            zipcode_master["Total"][row] = pop_total
            zipcode_master["White"][row] = white_total
            zipcode_master["Black"][row] = black_total
            
            ## if data is updated, the column notes the change
            zipcode_master["Updated"][row] = "Yes"
        except:
            
            ## if search gives no result, iteration moves to next row
            zipcode_master["Updated"][row] = "No"
            row = row + 1
            
    ## if row does not have null value, iteration moves to next row       
    else:
        zipcode_master["Updated"][row] = "No"
        row = row + 1

<ipython-input-14-fe4ed2d64003>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zipcode_master["Updated"][row] = "No"
<ipython-input-14-fe4ed2d64003>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zipcode_master["Updated"][row] = "No"


In [15]:
## displays updated dataframe
zipcode_master.head(10)

,Zipcode,Total,Black,White,NotHisp_Latino,Hisp_Latino,geoid,Median_Income,Below_Poverty_Percent,Below_Poverty_Total,Updated
0,32003,28676,2248,25726,26761,1915,86000US32003,111651,3.81,1089,No
1,32004,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,No
2,32006,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,No
3,32007,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,No
4,32008,4425,59,4225,3997,428,86000US32008,42172,12.02,529,No
5,32009,3661,50,3633,3544,117,86000US32009,90500,4.22,154,No
6,32011,14509,567,13914,14395,114,86000US32011,71701,9.28,1338,No
7,32013,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,No
8,32024,19452,1083,17863,18063,1389,86000US32024,74137,8.35,1617,No
9,32025,23416,5751,17053,21525,1891,86000US32025,59197,14.07,2665,No


### Exporting complete master dataframe as CSV file

In [16]:
zipcode_master.to_csv("zipcode_finalmaster.csv", index = False)

### Example: Generating zipcode demographics for Restaurant Relief Funds data

In [17]:
## importing restaurant relief funds data
rrf_data = pd.read_csv("rrf_data.csv")
rrf_data.head(3)

,State,Zipcode,Recipients_Total,Grants_Total
0,Florida,32003,4,388781.20
1,Florida,32008,1,19010.56
2,Florida,32011,1,36330.00


In [18]:
## joining demographic master sheet to rrf data
rrf_master = pd.merge(rrf_data, zipcode_master, on = "Zipcode", how = "left")

## displays updated dataframe
rrf_master.head(10)

,State,Zipcode,Recipients_Total,Grants_Total,Total,Black,White,NotHisp_Latino,Hisp_Latino,geoid,Median_Income,Below_Poverty_Percent,Below_Poverty_Total,Updated
0,Florida,32003,4,388781.20,28676,2248,25726,26761,1915,86000US32003,111651,3.81,1089,No
1,Florida,32008,1,19010.56,4425,59,4225,3997,428,86000US32008,42172,12.02,529,No
2,Florida,32011,1,36330.00,14509,567,13914,14395,114,86000US32011,71701,9.28,1338,No
3,Florida,32024,3,161550.52,19452,1083,17863,18063,1389,86000US32024,74137,8.35,1617,No
4,Florida,32025,3,106791.72,23416,5751,17053,21525,1891,86000US32025,59197,14.07,2665,No
5,Florida,32033,1,6860.95,4676,900,3670,4262,414,86000US32033,69541,23.57,1102,No
6,Florida,32034,11,2536967.13,36395,2362,32999,34256,2139,86000US32034,98304,9.55,3459,No
7,Florida,32038,1,2013.00,10184,1041,9119,9651,533,86000US32038,49238,16.13,1643,No
8,Florida,32043,2,72298.76,26743,2774,23472,24331,2412,86000US32043,76895,10.8,2810,No
9,Florida,32052,1,70484.59,9427,3945,5495,8753,674,86000US32052,52128,25.03,1535,No


In [19]:
## exporting to a spreadsheet
rrf_master.to_csv("rrf_zipcode_master.csv", index = False)